
# File Data Extraction
This notebook focuses on building a CSV file out of the raw file structure on disk in a given directory

In [283]:
# Import required libraries
import pandas as pd
import os

In [284]:
# Common source file extensions. This list is very incomplete. Intentionally not including JSON / XML
source_extensions = [
    '.cs', 
    '.vb', 
    '.java', 
    '.r', 
    '.agc', 
    '.fs', 
    '.js', 
    '.cpp', 
    '.go', 
    '.aspx', 
    '.jsp', 
    '.do', 
    '.php', 
    '.ipynb', 
    '.sh', 
    '.html', 
    '.lua', 
    '.css',
    '.md',
    '.sln'
    ]

def is_source_file(file_label):
    """
    Defines what a source file is.
    """
    file, _, _ = file_label
    _, ext = os.path.splitext(file)

    return ext.lower() in source_extensions

In [285]:
def count_lines(path):
    """
    Reads the file at the specified path and returns the number of lines in that file
    Code taken from https://stackoverflow.com/questions/845058/how-to-get-line-count-of-a-large-file-cheaply-in-python
    """

    def _make_gen(reader):
        b = reader(2 ** 16)
        while b:
            yield b
            b = reader(2 ** 16)

    with open(path, "rb") as f:
        count = sum(buf.count(b"\n") for buf in _make_gen(f.raw.read))

    return count

In [286]:
def get_file_metrics(files, root):
    """
    This function gets all metrics for the files and returns them as a list of file detail objects
    """
    results = []

    for file, folders, relative_path in files:
        lines = count_lines(file) # Slow as it actually reads the file
        _, filename = os.path.split(file)
        _, ext = os.path.splitext(filename)

        fullpath = ''
        area = 'root'

        if folders != None and len(folders) > 0:
            project = folders[0]

            for folder in folders[1:]:
                print(folder)
                if len(fullpath) > 0:
                    fullpath += '/'
                fullpath += folder
        else:
            project = ''

        if len(fullpath) <= 0:
            fullpath = '.'

        id = root + project + '/' + fullpath + '/' + filename

        if relative_path == '':
            rel_path = filename
        else:
            rel_path = relative_path + filename

        file_details = {
                        'fullpath': id,
                        'root': root,
                        'project': project,
                        'path': fullpath,
                        'area': area,
                        'relative_path': rel_path,
                        'filename': filename,
                        'ext': ext,
                        'lines': lines,
                        }
        results.append(file_details)

    return results

In [287]:
# Some directories should be ignored as their output is not helpful
ignored_directories = ['.git', '.vs', 'obj', 'ndependout', 'bin', 'debug']

def get_file_list(dir_path, relative_path, paths=None):
    """
    Gets a list of files in this this directory and all contained directories
    """

    files = list()
    contents = os.listdir(dir_path)

    for entry in contents:
        path = os.path.join(dir_path, entry)
        if os.path.isdir(path):
            # Ignore build and reporting directories
            if entry.lower() in ignored_directories:
                continue

            if relative_path == '':
                new_path = entry
            else:
                new_path = relative_path + '\\' + entry

            # Maintain an accurate, but separate, hierarchy array
            if paths is None:
                p = [entry]
            else:
                p = paths[:]

            files = files + get_file_list(path, new_path, p)
        else:
            files.append((path, paths, relative_path))
    return files

In [288]:
def get_source_file_metrics(root):
    """
    This function gets all source files and metrics associated with them from a given path
    """
    source_files = filter(is_source_file, get_file_list(root, ''))
    return get_file_metrics(list(source_files), root)

In [289]:
# Pull Source file metrics
files = []
root = 'C:/Dev/MachineLearning/'

files = get_source_file_metrics(root)

print(str(len(files)) + " files read")

.github
build
docs
eng
src
test
tools-local
1875 files read


In [295]:
# Grab the file metrics for source files and put them into a data frame
df = pd.DataFrame(files)
df = df.sort_values('lines', ascending=False)

def extract_area(row):
    rel_path = row['relative_path']

    row['area'] = 'Root'
    if not rel_path == None and rel_path.__contains__('\\'):
        parts = rel_path.split('\\')
        if len(parts) > 2:
            row['area'] = parts[1]
    return row

df = df.apply(extract_area, axis=1)

# Preview the data
df.head()

,fullpath,root,project,path,area,relative_path,filename,ext,lines
392,C:/Dev/MachineLearning/src/./PrimitiveDataFram...,C:/Dev/MachineLearning/,src,.,Microsoft.Data.AnalysisPrimitiveDataFrameColum...,src\Microsoft.Data.AnalysisPrimitiveDataFrameC...,PrimitiveDataFrameColumn.BinaryOperationAPIs.E...,.cs,15886
395,C:/Dev/MachineLearning/src/./PrimitiveDataFram...,C:/Dev/MachineLearning/,src,.,Microsoft.Data.AnalysisPrimitiveDataFrameColum...,src\Microsoft.Data.AnalysisPrimitiveDataFrameC...,PrimitiveDataFrameColumn.BinaryOperators.cs,.cs,7397
1618,C:/Dev/MachineLearning/test/./TestEntryPoints.cs,C:/Dev/MachineLearning/,test,.,Microsoft.ML.Core.Tests,test\Microsoft.ML.Core.Tests\UnitTestsTestEntr...,TestEntryPoints.cs,.cs,6904
400,C:/Dev/MachineLearning/src/./PrimitiveDataFram...,C:/Dev/MachineLearning/,src,.,Microsoft.Data.AnalysisPrimitiveDataFrameColum...,src\Microsoft.Data.AnalysisPrimitiveDataFrameC...,PrimitiveDataFrameColumnArithmetic.cs,.cs,6863
1194,C:/Dev/MachineLearning/src/./OnnxMl.cs,C:/Dev/MachineLearning/,src,.,Microsoft.ML.OnnxConverterOnnxMl.cs,src\Microsoft.ML.OnnxConverterOnnxMl.cs,OnnxMl.cs,.cs,5945


In [290]:
# Write to a file for other analyses processes
df.to_csv('FileSizes.csv')

,fullpath,root,project,path,area,relative_path,filename,ext,lines
392,C:/Dev/MachineLearning/src/./PrimitiveDataFram...,C:/Dev/MachineLearning/,src,.,root,src\Microsoft.Data.AnalysisPrimitiveDataFrameC...,PrimitiveDataFrameColumn.BinaryOperationAPIs.E...,.cs,15886
395,C:/Dev/MachineLearning/src/./PrimitiveDataFram...,C:/Dev/MachineLearning/,src,.,root,src\Microsoft.Data.AnalysisPrimitiveDataFrameC...,PrimitiveDataFrameColumn.BinaryOperators.cs,.cs,7397
1618,C:/Dev/MachineLearning/test/./TestEntryPoints.cs,C:/Dev/MachineLearning/,test,.,root,test\Microsoft.ML.Core.Tests\UnitTestsTestEntr...,TestEntryPoints.cs,.cs,6904
400,C:/Dev/MachineLearning/src/./PrimitiveDataFram...,C:/Dev/MachineLearning/,src,.,root,src\Microsoft.Data.AnalysisPrimitiveDataFrameC...,PrimitiveDataFrameColumnArithmetic.cs,.cs,6863
1194,C:/Dev/MachineLearning/src/./OnnxMl.cs,C:/Dev/MachineLearning/,src,.,root,src\Microsoft.ML.OnnxConverterOnnxMl.cs,OnnxMl.cs,.cs,5945
